In [7]:
import requests
import pandas as pd
import json

# URL of the JSON data
url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/tour_occ_nim?format=JSON&sinceTimePeriod=2012-01&geo=DK&geo=DE&geo=EL&geo=ES&geo=HR&geo=IT&geo=PT&geo=FI&geo=SE&geo=NO&unit=NR&unit=PCH_SM&unit=PCH_SM_19&c_resid=DOM&c_resid=FOR&nace_r2=I551&nace_r2=I552&nace_r2=I553&lang=de"

# Download and parse the JSON
response = requests.get(url)
data = response.json()
dims = data['dimension']
values = data['value']

# Extract dimension metadata
dim_order = data['id']  # dimension order
dim_sizes = data['size']  # sizes of dimensions
dim_labels = {dim: dims[dim]['label'] for dim in dim_order}
dim_categories = {dim: dims[dim]['category']['label'] for dim in dim_order}
dim_category_keys = {dim: list(dims[dim]['category']['label'].keys()) for dim in dim_order}

# Flatten values
records = []
for idx, val in values.items():
    idx = int(idx)
    indexes = []
    remainder = idx
    for size in reversed(dim_sizes):
        indexes.append(remainder % size)
        remainder //= size
    indexes.reverse()

    # Map to dimension keys and values
    row = {}
    for i, dim in enumerate(dim_order):
        keys = dim_category_keys[dim]
        key = keys[indexes[i]]
        label = dim_categories[dim][key]
        dim_name = dim_labels[dim]

        # Use your specified column names
        if dim == 'freq':
            row['Zeitliche_Frequenz_Idx'] = key
            row['Zeitliche_Frequenz'] = label
        elif dim == 'c_resid':
            row['Aufenthaltsland_Idx'] = key
            row['Aufenthaltsland'] = label
        elif dim == 'unit':
            row['Maßeinheit_Idx'] = key
            row['Maßeinheit'] = label
        elif dim == 'nace_r2':
            row['NACEr2_Idx'] = key
            row['NACEr2'] = label
        elif dim == 'geo':
            row['Geopolitische_Meldeeinheit_Idx'] = key
            row['Geopolitische_Meldeeinheit'] = label
        elif dim == 'time':
            # row['Zeit_Idx'] = key
            row['JahrMonat'] = label
    row['value'] = val
    records.append(row)

# Create DataFrame
df = pd.DataFrame(records)

# Export to CSV
csv_filename = "estat_tour_overnight_stays_2012-2025_eu10_de.csv"
df.to_csv(csv_filename, index=False)

print(f"Exported to {csv_filename}")
df.head()


Exported to estat_tour_overnight_stays_2012-2025_eu10_de.csv


,Zeitliche_Frequenz_Idx,Zeitliche_Frequenz,Aufenthaltsland_Idx,Aufenthaltsland,Maßeinheit_Idx,Maßeinheit,NACEr2_Idx,NACEr2,Geopolitische_Meldeeinheit_Idx,Geopolitische_Meldeeinheit,JahrMonat,value
0,M,Monatlich,DOM,Inland,NR,Anzahl,I551,"Hotels, Gasthöfe und Pensionen",DE,Deutschland,2012-01,11022604.0
1,M,Monatlich,DOM,Inland,NR,Anzahl,I551,"Hotels, Gasthöfe und Pensionen",DE,Deutschland,2012-02,11916641.0
2,M,Monatlich,DOM,Inland,NR,Anzahl,I551,"Hotels, Gasthöfe und Pensionen",DE,Deutschland,2012-03,14018007.0
3,M,Monatlich,DOM,Inland,NR,Anzahl,I551,"Hotels, Gasthöfe und Pensionen",DE,Deutschland,2012-04,15472920.0
4,M,Monatlich,DOM,Inland,NR,Anzahl,I551,"Hotels, Gasthöfe und Pensionen",DE,Deutschland,2012-05,18319333.0
